# Parse and Consolidate Agent Evaluation Data

This notebook takes the raw CSV export containing span data and manually added annotations from a Phoenix UI Dataset, parses the relevant information, and consolidates it into a final DataFrame suitable for evaluation.

## 1. Setup and Imports

Import necessary libraries (pandas, json, etc.) and configure display options for pandas DataFrames.

In [1]:
import pandas as pd
import json
from pathlib import Path
import ast # For safely evaluating literal strings if needed

# Configure pandas display options
pd.set_option('display.max_rows', 50) # Show more rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.width', 2000) # Wider display
pd.set_option('display.max_colwidth', 300) # Show more text in columns

## 2. Load Raw Data

Load the exported CSV file into a pandas DataFrame (`df_raw`) and perform an initial inspection (info, head) to understand its structure.

In [2]:
# Define the path to the CSV file
CSV_FILENAME = "Transcipt-Agent-annotated 2025-04-27T22_47_27.666Z-3.csv" # Note the typo in "Transcipt"
DATA_DIR = Path("./data") # Relative to notebook location
CSV_PATH = DATA_DIR / CSV_FILENAME

print(f"Attempting to load CSV: {CSV_PATH}")

# Check if file exists before loading
if not CSV_PATH.is_file():
    print(f"Error: CSV file not found at {CSV_PATH}")
    # List directory contents if file not found for debugging
    print(f"\nContents of {DATA_DIR} (if it exists):")
    try:
        for item in DATA_DIR.iterdir():
            print(f"- {item.name}")
    except FileNotFoundError:
        print(f"  Error: Directory {DATA_DIR} not found.")
    # Raise an error to stop execution
    raise FileNotFoundError(f"CSV file not found: {CSV_PATH}")

# Load the CSV
try:
    df_raw = pd.read_csv(CSV_PATH)
    print("CSV loaded successfully.")
    # Display basic info and head after loading
    print("\n--- Raw DataFrame Info ---")
    df_raw.info()
    print("\n--- Raw DataFrame Head (first 10 rows) ---")
    display(df_raw.head(10))
    print("\n--- Raw Column Names ---")
    print(df_raw.columns.tolist())
except Exception as e:
    print(f"Error loading CSV: {e}")
    # Raise an error to stop execution
    raise e


Attempting to load CSV: data/Transcipt-Agent-annotated 2025-04-27T22_47_27.666Z-3.csv
CSV loaded successfully.

--- Raw DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            125 non-null    object
 1   input_messages        36 non-null     object
 2   input_tools           19 non-null     object
 3   output_messages       36 non-null     object
 4   metadata_span_kind    125 non-null    object
 5   metadata_annotations  38 non-null     object
 6   input_input           19 non-null     object
 7   output_output         19 non-null     object
dtypes: object(8)
memory usage: 7.9+ KB

--- Raw DataFrame Head (first 10 rows) ---


,example_id,input_messages,input_tools,output_messages,metadata_span_kind,metadata_annotations,input_input,output_output
0,RGF0YXNldEV4YW1wbGU6MTUw,"[{'role': 'system', 'content': ""\n You are a helpful assistant designed to answer questions about the LearnAIWithAI Workshop 1 transcript. \n Use the available tools to query the transcript database when necessary. \n The database table is ''transcript_segments'' and contain...","[{'type': 'function', 'function': {'name': 'query_database', 'description': 'Executes a read-only SQL query against a database containing transcript segments from Workshop 1. Use this to find specific information mentioned in the workshop transcript.', 'parameters': {'type': 'object', 'propertie...","[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT * FROM transcript_segments WHERE text LIKE \'%Jeff Pidcock%\'""}'}}]}]",LLM,"{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': ...",NaN,NaN
1,RGF0YXNldEV4YW1wbGU6MTUx,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
2,RGF0YXNldEV4YW1wbGU6MTUy,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
3,RGF0YXNldEV4YW1wbGU6MTUz,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
4,RGF0YXNldEV4YW1wbGU6MTU0,"[{'role': 'system', 'content': ""\n You are a helpful assistant designed to answer questions about the LearnAIWithAI Workshop 1 transcript. \n Use the available tools to query the transcript database when necessary. \n The database table is ''transcript_segments'' and contain...",NaN,"[{'role': 'assistant', 'content': 'I cannot answer the question about who Jeff Pidcock is based on the available transcript data.'}]",LLM,NaN,NaN,NaN
5,RGF0YXNldEV4YW1wbGU6MTU1,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
6,RGF0YXNldEV4YW1wbGU6MTU2,NaN,NaN,NaN,UNKNOWN,"{'Final Answer Quality': {'label': 'Fail', 'score': 0.0, 'explanation': 'The agent provided a factually incorrect final answer (""I cannot answer..."") because the underlying SQL query failed to retrieve the available information from the database.', 'metadata': {}, 'annotator_kind': 'HUMAN'}}",Who is Jeff Pidcock?,I cannot answer the question about who Jeff Pidcock is based on the available transcript data.
7,RGF0YXNldEV4YW1wbGU6MTU3,"[{'role': 'system', 'content': ""\n You are a helpful assistant designed to answer questions about the LearnAIWithAI Workshop 1 transcript. \n Use the available tools to query the transcript database when necessary. \n The database table is ''transcript_segments'' and contain...","[{'type': 'function', 'function': {'name': 'query_database', 'description': 'Executes a read-only SQL query against a database containing transcript segments from Workshop 1. Use this to find specific information mentioned in the workshop transcript.', 'parameters': {'type': 'object', 'propertie...","[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT text FROM transcript_segments WHERE speaker=\'Stefan Krawczyk\' AND text LIKE \'%introduction%\'""}'}}]}]",LLM,"{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': ' The query asks for specific spoken content attributed to a speaker, requiring a database lookup.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': 0.0, 'explanation': ""Th...",NaN,NaN
8,RGF0YXNldEV4YW1wbGU6MTU4,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN
9,RGF0YXNldEV4YW1wbGU6MTU5,NaN,NaN,NaN,UNKNOWN,NaN,NaN,NaN



--- Raw Column Names ---
['example_id', 'input_messages', 'input_tools', 'output_messages', 'metadata_span_kind', 'metadata_annotations', 'input_input', 'output_output']


## 3. Parse Annotations

The `metadata_annotations` column contains JSON strings with potentially multiple annotation types (Final Answer Quality, SQL Correctness, Tool Usage Correctness).

This section defines a function to parse this JSON and extracts the 'label' for each annotation type found into new columns in the `df_raw` DataFrame. It also verifies the extraction using value counts.

In [3]:
# Goal: Parse the string containing annotations into a Python dictionary.
# Note: The data looks like a Python dict literal (e.g., using single quotes)
# rather than strict JSON. We'll use ast.literal_eval for this.

def parse_annotation_string(anno_text):
    """Safely parses a string representation of a Python dict."""
    if pd.isna(anno_text) or not isinstance(anno_text, str):
        return None
    try:
        # ast.literal_eval safely evaluates a string containing a Python literal
        # (like dicts, lists, numbers, strings, booleans, None)
        parsed_dict = ast.literal_eval(anno_text)
        if isinstance(parsed_dict, dict):
             return parsed_dict
        else:
             # Handle cases where the string parses but isn't a dict
             # print(f"Warning: Parsed to non-dict type ({type(parsed_dict)}): {anno_text[:100]}...")
             return None
    except (ValueError, SyntaxError, TypeError) as e:
        # If parsing fails (e.g., malformed string)
        # print(f"Warning: Failed to parse annotation string: {anno_text[:100]}... Error: {e}")
        return None

# --- Test the function on a sample ---
sample_annotation = df_raw['metadata_annotations'].dropna().iloc[0] # Get first non-NA value
print("Sample Raw String:", sample_annotation)
print("Parsed Sample:", parse_annotation_string(sample_annotation))

# --- Apply to the whole column ---
df_raw['parsed_annotations'] = df_raw['metadata_annotations'].apply(parse_annotation_string)
print(f"\nApplied parsing. Found {df_raw['parsed_annotations'].notna().sum()} successfully parsed annotations.")

# --- Display head with parsed data ---
print("\n--- Head with parsed_annotations column ---")
display(df_raw[['metadata_annotations', 'parsed_annotations']].head())


Sample Raw String: {'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': 0.0, 'explanation': 'The specific SQL query (LIKE \'%Jeff Pidcock%\') failed functionally. It did not retrieve the existing mention of "Jeff Pidcock" from the transcript, most likely due to case sensitivity, making it an incorrect implementation for the task.', 'metadata': {}, 'annotator_kind': 'HUMAN'}}
Parsed Sample: {'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': 0.0, 'explanation': 'The specific SQL query (LIKE \'%Jeff Pidcock%\'

,metadata_annotations,parsed_annotations
0,"{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': ...","{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': ..."
1,NaN,None
2,NaN,None
3,NaN,None
4,NaN,None


In [4]:
# Goal: Extract the specific 'label' value from a parsed annotation dictionary.
# Example structure: {'Final Answer Quality': {'label': 'Pass', 'score': 1, ...}}

def get_label(parsed_dict, annotation_key):
    """Safely extracts the 'label' from a nested dictionary."""
    if isinstance(parsed_dict, dict):
        annotation_details = parsed_dict.get(annotation_key) # Get the inner dict
        if isinstance(annotation_details, dict):
            return annotation_details.get('label') # Get the label
    return None # Return None if structure is wrong or keys missing

# --- Test the function ---
sample_parsed = df_raw['parsed_annotations'].dropna().iloc[0]
print("Sample Parsed Dict:", sample_parsed)
print("Extracted 'Final Answer Quality' Label:", get_label(sample_parsed, 'Final Answer Quality'))
print("Extracted 'SQL Correctness' Label:", get_label(sample_parsed, 'SQL Correctness')) # Might be None if not present

Sample Parsed Dict: {'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': 0.0, 'explanation': 'The specific SQL query (LIKE \'%Jeff Pidcock%\') failed functionally. It did not retrieve the existing mention of "Jeff Pidcock" from the transcript, most likely due to case sensitivity, making it an incorrect implementation for the task.', 'metadata': {}, 'annotator_kind': 'HUMAN'}}
Extracted 'Final Answer Quality' Label: None
Extracted 'SQL Correctness' Label: Incorrect


## 4. Calculate Summary Metrics

Calculate overall scores (e.g., correctness percentages) for each annotation type, similar to the overview metrics shown in the Phoenix UI. This involves:
*   Dropping examples that were not annotated (`None` values).
*   Mapping text labels ('Correct', 'Pass', 'Fail') to numerical scores (1 or 0).
*   Calculating the mean score for annotated examples.

In [5]:
# Goal: Create new columns for each annotation label.

# Define the keys we expect
annotation_keys = ['Final Answer Quality', 'SQL Correctness', 'Tool Usage Correctness']

print("Creating label columns:")
# Explicitly create each column
df_raw['final_answer_quality_label'] = df_raw['parsed_annotations'].apply(lambda x: get_label(x, 'Final Answer Quality'))
print("- Created final_answer_quality_label")

df_raw['sql_correctness_label'] = df_raw['parsed_annotations'].apply(lambda x: get_label(x, 'SQL Correctness'))
print("- Created sql_correctness_label")

df_raw['tool_usage_correctness_label'] = df_raw['parsed_annotations'].apply(lambda x: get_label(x, 'Tool Usage Correctness'))
print("- Created tool_usage_correctness_label")


# --- Verify with value counts ---
print("\n--- Value Counts for Labels ---")

print("\nFinal Answer Quality Labels:")
print(df_raw['final_answer_quality_label'].value_counts(dropna=False))

print("\nSQL Correctness Labels:")
print(df_raw['sql_correctness_label'].value_counts(dropna=False))

print("\nTool Usage Correctness Labels:")
print(df_raw['tool_usage_correctness_label'].value_counts(dropna=False))

# --- Display final relevant columns ---
print("\n--- Head showing extracted labels ---")
display(df_raw[['input_input', 'output_output', 'final_answer_quality_label', 'sql_correctness_label', 'tool_usage_correctness_label']].head(10))


Creating label columns:
- Created final_answer_quality_label
- Created sql_correctness_label
- Created tool_usage_correctness_label

--- Value Counts for Labels ---

Final Answer Quality Labels:
final_answer_quality_label
None       106
Pass        11
Fail         6
Correct      1
Correcr      1
Name: count, dtype: int64

SQL Correctness Labels:
sql_correctness_label
None         108
Correct       11
Incorrect      6
Name: count, dtype: int64

Tool Usage Correctness Labels:
tool_usage_correctness_label
None       106
Correct     18
Correcr      1
Name: count, dtype: int64

--- Head showing extracted labels ---


,input_input,output_output,final_answer_quality_label,sql_correctness_label,tool_usage_correctness_label
0,NaN,NaN,None,Incorrect,Correct
1,NaN,NaN,None,None,None
2,NaN,NaN,None,None,None
3,NaN,NaN,None,None,None
4,NaN,NaN,None,None,None
5,NaN,NaN,None,None,None
6,Who is Jeff Pidcock?,I cannot answer the question about who Jeff Pidcock is based on the available transcript data.,Fail,None,None
7,NaN,NaN,None,Incorrect,Correct
8,NaN,NaN,None,None,None
9,NaN,NaN,None,None,None


    ## 5. Calculate Summary Metrics

    Now that we have parsed the annotations into separate columns, let's calculate summary scores for each annotation type. We will:
    *   Exclude examples that were not annotated (i.e., drop `None` values).
    *   Convert the text labels ('Pass', 'Correct', 'Fail', etc.) into numerical scores (1 for success, 0 for failure).
    *   Calculate the average score for each metric based only on the annotated examples.
    *   Print these scores, formatted similarly to the overview metrics seen in the Phoenix UI.

In [6]:
    print("\\n--- Evaluation Summary Metrics ---")

    # --- Tool Usage Correctness ---
    tool_usage_labels = df_raw['tool_usage_correctness_label'].dropna() # Drop None/NaN values
    if not tool_usage_labels.empty:
        # Handle potential typo 'Correcr' -> 'Correct' just in case
        tool_usage_labels_clean = tool_usage_labels.replace('Correcr', 'Correct')
        # Map 'Correct' to 1, others (implicitly Incorrect if they existed) to 0
        tool_usage_scores = tool_usage_labels_clean.map({'Correct': 1}).fillna(0) # Assume only 'Correct' maps to 1
        tool_usage_mean = tool_usage_scores.mean()
        print(f"Tool Usage Correctness Score: {tool_usage_mean:.2f} (from {len(tool_usage_labels)} annotated examples)")
    else:
        print("Tool Usage Correctness Score: N/A (no annotated examples)")


    # --- SQL Correctness ---
    sql_labels = df_raw['sql_correctness_label'].dropna() # Drop None/NaN values
    if not sql_labels.empty:
        # Map 'Correct' to 1, 'Incorrect' to 0
        sql_scores = sql_labels.map({'Correct': 1, 'Incorrect': 0})
        # Check for unmapped values (e.g., if there were other labels)
        if sql_scores.isna().any():
             print(f"  Warning: Unmapped SQL Correctness labels found: {sql_labels[sql_scores.isna()].unique()}")
        sql_mean = sql_scores.mean() # NaNs were dropped by .dropna()
        print(f"SQL Correctness Score: {sql_mean:.2f} (from {len(sql_labels)} annotated examples)")
    else:
        print("SQL Correctness Score: N/A (no annotated examples)")

    # --- Final Answer Quality ---
    final_answer_labels = df_raw['final_answer_quality_label'].dropna() # Drop None/NaN values
    if not final_answer_labels.empty:
         # Handle potential typo 'Correcr' -> 'Correct'
        final_answer_labels_clean = final_answer_labels.replace('Correcr', 'Correct')
        # Map 'Pass' and 'Correct' to 1 (positive quality), 'Fail' to 0
        final_answer_scores = final_answer_labels_clean.map({'Pass': 1, 'Correct': 1, 'Fail': 0})
         # Check for unmapped values
        if final_answer_scores.isna().any():
             print(f"  Warning: Unmapped Final Answer Quality labels found: {final_answer_labels_clean[final_answer_scores.isna()].unique()}")
        final_answer_mean = final_answer_scores.mean() # NaNs were dropped by .dropna()
        print(f"Final Answer Quality Score: {final_answer_mean:.2f} (from {len(final_answer_labels)} annotated examples)")
    else:
        print("Final Answer Quality Score: N/A (no annotated examples)")

\n--- Evaluation Summary Metrics ---
Tool Usage Correctness Score: 1.00 (from 19 annotated examples)
SQL Correctness Score: 0.65 (from 17 annotated examples)
Final Answer Quality Score: 0.68 (from 19 annotated examples)


## 5. Extract Generated SQL

Define a function to parse the `output_messages` column (which contains message history including tool calls) and extract the SQL query generated by the `query_database` tool. Apply this function to create the `generated_sql` column.

In [7]:
# --- Extract Generated SQL ---

def extract_sql_from_output(messages_str):
    """Parses the 'output_messages' string to find the generated SQL query."""
    if pd.isna(messages_str) or not isinstance(messages_str, str):
        return None
    try:
        # It's a string representation of a list of dicts
        messages = ast.literal_eval(messages_str)
        if not isinstance(messages, list):
            return None

        for message in messages:
            # Look for the assistant's message containing tool calls
            if message.get('role') == 'assistant' and 'tool_calls' in message:
                tool_calls = message['tool_calls']
                if isinstance(tool_calls, list):
                    for tool_call in tool_calls:
                        # Check if it's the query_database tool
                        if isinstance(tool_call, dict) and tool_call.get('function', {}).get('name') == 'query_database':
                            # Arguments might be a JSON string within the structure
                            arguments_str = tool_call.get('function', {}).get('arguments')
                            if isinstance(arguments_str, str):
                                try:
                                    arguments = json.loads(arguments_str) # Parse the arguments JSON
                                    return arguments.get('sql_query') # Extract the SQL query
                                except json.JSONDecodeError:
                                    continue # Ignore if arguments aren't valid JSON
                            # Handle case where arguments might already be a dict (less likely from log)
                            elif isinstance(arguments_str, dict):
                                 return arguments_str.get('sql_query')

    except (ValueError, SyntaxError, TypeError) as e:
        # print(f"Warning: Could not parse messages string: {messages_str[:100]}... Error: {e}")
        return None
    return None # Return None if no relevant tool call found

# Apply the function to the 'output_messages' column
df_raw['generated_sql'] = df_raw['output_messages'].apply(extract_sql_from_output)

# --- Display results ---
print("\\n--- Head showing extracted SQL (for non-null SQL) ---")
sql_cols = ['example_id', 'output_messages', 'generated_sql']

# Filter for rows where SQL was potentially generated (output_messages is not NaN)
# and where the parsing function actually found SQL
sql_extracted_df = df_raw[df_raw['generated_sql'].notna()]

if not sql_extracted_df.empty:
    display(sql_extracted_df[sql_cols].head())
else:
    print("No rows with extracted SQL found.")


print(f"\\nFound {df_raw['generated_sql'].notna().sum()} rows with extracted SQL.")


\n--- Head showing extracted SQL (for non-null SQL) ---


,example_id,output_messages,generated_sql
0,RGF0YXNldEV4YW1wbGU6MTUw,"[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT * FROM transcript_segments WHERE text LIKE \'%Jeff Pidcock%\'""}'}}]}]",SELECT * FROM transcript_segments WHERE text LIKE '%Jeff Pidcock%'
7,RGF0YXNldEV4YW1wbGU6MTU3,"[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT text FROM transcript_segments WHERE speaker=\'Stefan Krawczyk\' AND text LIKE \'%introduction%\'""}'}}]}]",SELECT text FROM transcript_segments WHERE speaker='Stefan Krawczyk' AND text LIKE '%introduction%'
14,RGF0YXNldEV4YW1wbGU6MTY0,"[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT DISTINCT speaker FROM transcript_segments""}'}}]}]",SELECT DISTINCT speaker FROM transcript_segments
21,RGF0YXNldEV4YW1wbGU6MTcx,"[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT SUM(word_count) AS total_words FROM transcript_segments WHERE speaker = \'Hugo\'""}'}}]}]",SELECT SUM(word_count) AS total_words FROM transcript_segments WHERE speaker = 'Hugo'
28,RGF0YXNldEV4YW1wbGU6MTc4,"[{'role': 'assistant', 'tool_calls': [{'function': {'name': 'query_database', 'arguments': '{""sql_query"":""SELECT start_time_seconds, end_time_seconds, text FROM transcript_segments WHERE text LIKE \'%evaluation%\'""}'}}]}]","SELECT start_time_seconds, end_time_seconds, text FROM transcript_segments WHERE text LIKE '%evaluation%'"


\nFound 17 rows with extracted SQL.


## 6. Consolidate Data into Final Evaluation DataFrame (`df_eval`)

Create the final `df_eval` DataFrame where each row represents a single agent interaction (trace).

This step leverages the observed pattern that the root span (containing `input_input` and `output_output`) consistently appears 6 rows *after* the relevant LLM span (containing `generated_sql` and annotations like Tool Usage/SQL Correctness) in the CSV export.

It iterates through the root spans, grabs data from the corresponding LLM span (at index N-6), combines the information (including labels *and* explanations), and builds the final table.

In [8]:
# --- Find Trace ID ---

# Let's inspect the structure of input_messages and output_messages
# on rows where they are not null, to find the trace_id

print("--- Inspecting input_messages (first non-null) ---")
first_input_msg_str = df_raw['input_messages'].dropna().iloc[0]
if isinstance(first_input_msg_str, str):
    try:
        first_input_msg = ast.literal_eval(first_input_msg_str)
        print(json.dumps(first_input_msg, indent=2)) # Pretty print the structure
    except Exception as e:
        print(f"Could not parse input_messages: {e}")
        print(f"Raw string: {first_input_msg_str[:500]}...")
else:
    print("First input_messages entry is not a string or is null.")


print("\n--- Inspecting output_messages (first non-null) ---")
first_output_msg_str = df_raw['output_messages'].dropna().iloc[0]
if isinstance(first_output_msg_str, str):
    try:
        first_output_msg = ast.literal_eval(first_output_msg_str)
        print(json.dumps(first_output_msg, indent=2)) # Pretty print the structure
    except Exception as e:
        print(f"Could not parse output_messages: {e}")
        print(f"Raw string: {first_output_msg_str[:500]}...")
else:
    print("First output_messages entry is not a string or is null.")

# Also check if example_id might contain trace_id implicitly
print("\n--- Example ID format ---")
print(df_raw['example_id'].head())
# The format RGF0YX... looks like Base64 encoding.
# Let's try decoding one to see if it contains useful info like trace_id
import base64
try:
    decoded_id = base64.b64decode(df_raw['example_id'].iloc[0]).decode('utf-8')
    print(f"\nDecoded first example_id: {decoded_id}")
    # Example format might be "DatasetExample:<trace_id>:<span_id>" or similar
except Exception as e:
    print(f"\nCould not decode example_id as Base64: {e}")


--- Inspecting input_messages (first non-null) ---
[
  {
    "role": "system",
    "content": "\n        You are a helpful assistant designed to answer questions about the LearnAIWithAI Workshop 1 transcript. \n        Use the available tools to query the transcript database when necessary. \n        The database table is ''transcript_segments'' and contains segments of the transcript.\n        Base your answers SOLELY on the information retrieved from the database using the tools. \n        If the information is not found in the database, say that you cannot answer the question based on the available transcript data.\n        Be concise and directly answer the user's query based on the tool results.\n        "
  },
  {
    "role": "user",
    "content": "Who is Jeff Pidcock?"
  }
]

--- Inspecting output_messages (first non-null) ---
[
  {
    "role": "assistant",
    "tool_calls": [
      {
        "function": {
          "name": "query_database",
          "arguments": "{\"sql_query

In [9]:
# --- Inspect Row Order ---

print("\\n--- Inspecting Row Order (First 40 Rows) ---")
# Select columns that help identify the span type and content
inspect_cols = [
    'example_id',
    'metadata_span_kind',
    'input_input',
    'output_output',
    'generated_sql',
    'parsed_annotations' # Or just the label columns if preferred
]

# Display a larger chunk of the DataFrame
# Use display() for better notebook formatting
display(df_raw[inspect_cols].head(40))

\n--- Inspecting Row Order (First 40 Rows) ---


,example_id,metadata_span_kind,input_input,output_output,generated_sql,parsed_annotations
0,RGF0YXNldEV4YW1wbGU6MTUw,LLM,NaN,NaN,SELECT * FROM transcript_segments WHERE text LIKE '%Jeff Pidcock%',"{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': 'The agent correctly identified that answering this question requires querying the database to find mentions of the name.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': ..."
1,RGF0YXNldEV4YW1wbGU6MTUx,UNKNOWN,NaN,NaN,None,None
2,RGF0YXNldEV4YW1wbGU6MTUy,UNKNOWN,NaN,NaN,None,None
3,RGF0YXNldEV4YW1wbGU6MTUz,UNKNOWN,NaN,NaN,None,None
4,RGF0YXNldEV4YW1wbGU6MTU0,LLM,NaN,NaN,None,None
5,RGF0YXNldEV4YW1wbGU6MTU1,UNKNOWN,NaN,NaN,None,None
6,RGF0YXNldEV4YW1wbGU6MTU2,UNKNOWN,Who is Jeff Pidcock?,I cannot answer the question about who Jeff Pidcock is based on the available transcript data.,None,"{'Final Answer Quality': {'label': 'Fail', 'score': 0.0, 'explanation': 'The agent provided a factually incorrect final answer (""I cannot answer..."") because the underlying SQL query failed to retrieve the available information from the database.', 'metadata': {}, 'annotator_kind': 'HUMAN'}}"
7,RGF0YXNldEV4YW1wbGU6MTU3,LLM,NaN,NaN,SELECT text FROM transcript_segments WHERE speaker='Stefan Krawczyk' AND text LIKE '%introduction%',"{'Tool Usage Correctness': {'label': 'Correct', 'score': 1.0, 'explanation': ' The query asks for specific spoken content attributed to a speaker, requiring a database lookup.', 'metadata': {}, 'annotator_kind': 'HUMAN'}, 'SQL Correctness': {'label': 'Incorrect', 'score': 0.0, 'explanation': 'Th..."
8,RGF0YXNldEV4YW1wbGU6MTU4,UNKNOWN,NaN,NaN,None,None
9,RGF0YXNldEV4YW1wbGU6MTU5,UNKNOWN,NaN,NaN,None,None


In [11]:
# --- Consolidate Data based on N-6 Pattern (including Explanations) ---

final_data = []
# Ensure we have the 'parsed_annotations' column from Cell 3c
if 'parsed_annotations' not in df_raw.columns:
     raise ValueError("The 'parsed_annotations' column is missing. Please re-run Cell 3c.")

# Ensure we have the 'generated_sql' column from the SQL extraction cell
if 'generated_sql' not in df_raw.columns:
     raise ValueError("The 'generated_sql' column is missing. Please re-run the SQL extraction cell.")


root_indices = df_raw[df_raw['input_input'].notna()].index

print(f"Found {len(root_indices)} root spans. Consolidating data including explanations...")

for root_idx in root_indices:
    llm_idx = root_idx - 6

    if llm_idx < 0:
        print(f"  Warning: Skipping root span at index {root_idx} because index {llm_idx} is invalid.")
        continue

    # Get data from root span
    user_query = df_raw.loc[root_idx, 'input_input']
    final_answer = df_raw.loc[root_idx, 'output_output']
    root_annotations = df_raw.loc[root_idx, 'parsed_annotations']

    # Get data from the corresponding LLM span (6 rows above)
    llm_span = df_raw.loc[llm_idx]

    # Verify it's an LLM span
    if llm_span['metadata_span_kind'] != 'LLM':
         print(f"  Warning: Span at index {llm_idx} (for root {root_idx}) is not LLM type ({llm_span['metadata_span_kind']}). Skipping trace.")
         continue

    generated_sql = llm_span['generated_sql']
    llm_annotations = llm_span['parsed_annotations']

    # Combine annotations from both spans
    combined_annotations = {}
    if isinstance(llm_annotations, dict):
        combined_annotations.update(llm_annotations)
    if isinstance(root_annotations, dict):
        combined_annotations.update(root_annotations) # Root annotations overwrite LLM if keys conflict

    # Extract final labels AND explanations from the combined dictionary
    # Using .get() safely handles missing keys or nested structures
    tool_usage_label = combined_annotations.get('Tool Usage Correctness', {}).get('label')
    tool_usage_explanation = combined_annotations.get('Tool Usage Correctness', {}).get('explanation')

    sql_correctness_label = combined_annotations.get('SQL Correctness', {}).get('label')
    sql_correctness_explanation = combined_annotations.get('SQL Correctness', {}).get('explanation')

    final_answer_label = combined_annotations.get('Final Answer Quality', {}).get('label')
    final_answer_explanation = combined_annotations.get('Final Answer Quality', {}).get('explanation')

    # Add to our list
    final_data.append({
        'user_query': user_query,
        'final_answer': final_answer,
        'generated_sql': generated_sql,
        'tool_called': pd.notna(generated_sql), # True if SQL was generated
        'tool_usage_correctness_label': tool_usage_label,
        'sql_correctness_label': sql_correctness_label,
        'final_answer_quality_label': final_answer_label,
        'tool_usage_explanation': tool_usage_explanation,
        'sql_correctness_explanation': sql_correctness_explanation,
        'final_answer_explanation': final_answer_explanation,
        'root_span_idx': root_idx, # Keep index for reference
        'llm_span_idx': llm_idx    # Keep index for reference
    })

# Create the final DataFrame
df_eval = pd.DataFrame(final_data)

# Add safety checks for required columns existing from previous cells
required_cols = ['parsed_annotations', 'generated_sql']
for col in required_cols:
    if col not in df_raw.columns:
        print(f"ERROR: Required column '{col}' not found in df_raw. Please ensure the cell that creates it has been run.")
        # You might want to stop execution here or handle it differently
        raise KeyError(f"Missing required column: {col}")


print(f"Successfully created df_eval with {len(df_eval)} rows.")

# Display the first few rows of the consolidated DataFrame
print("\n--- Final Consolidated DataFrame (df_eval) with Explanations ---")
# Select columns to display - adjust as needed
display_cols = [
    'user_query', 'final_answer', 'generated_sql', 'tool_called',
    'tool_usage_correctness_label', 'sql_correctness_label', 'final_answer_quality_label',
    'tool_usage_explanation', 'sql_correctness_explanation', 'final_answer_explanation'
]
display(df_eval[display_cols].head())


Found 19 root spans. Consolidating data including explanations...
Successfully created df_eval with 17 rows.

--- Final Consolidated DataFrame (df_eval) with Explanations ---


,user_query,final_answer,generated_sql,tool_called,tool_usage_correctness_label,sql_correctness_label,final_answer_quality_label,tool_usage_explanation,sql_correctness_explanation,final_answer_explanation
0,Who is Jeff Pidcock?,I cannot answer the question about who Jeff Pidcock is based on the available transcript data.,SELECT * FROM transcript_segments WHERE text LIKE '%Jeff Pidcock%',True,Correct,Incorrect,Fail,The agent correctly identified that answering this question requires querying the database to find mentions of the name.,"The specific SQL query (LIKE '%Jeff Pidcock%') failed functionally. It did not retrieve the existing mention of ""Jeff Pidcock"" from the transcript, most likely due to case sensitivity, making it an incorrect implementation for the task.","The agent provided a factually incorrect final answer (""I cannot answer..."") because the underlying SQL query failed to retrieve the available information from the database."
1,What did Stefan Krawczyk say during his introduction?,I cannot answer the question based on the available transcript data.,SELECT text FROM transcript_segments WHERE speaker='Stefan Krawczyk' AND text LIKE '%introduction%',True,Correct,Incorrect,Fail,"The query asks for specific spoken content attributed to a speaker, requiring a database lookup.","The SQL relies on the keyword LIKE '%introduction%', which is naive. Stefan likely introduced himself without using that exact word. The query lacks semantic understanding and doesn't consider finding segments early in the transcript or associated with the speaker starting.\n","The agent failed to retrieve likely available information (Stefan's introduction) because the generated SQL query was semantically inadequate, thus failing to satisfy the user's query."
2,List all unique speakers mentioned.,The unique speakers mentioned in the transcript are:\n\n1. Hugo Bowne-Anderson\n2. Stefan Krawczyk\n3. GP\n4. William Horton\n5. Nathan Danielsen\n6. Greg\n7. Sanket Firodiya\n8. Philip Kiely\n9. Caleb Tutty\n10. Rafiolaverria,SELECT DISTINCT speaker FROM transcript_segments,True,Correct,Correct,Pass,The SQL query correctly uses SELECT DISTINCT speaker to retrieve the unique speaker names from the table.,The SQL query correctly uses SELECT DISTINCT speaker to retrieve the unique speaker names from the table.,"The answer accurately lists all the unique speakers retrieved by the SQL query, directly fulfilling the user's request"
3,How many words did Hugo speak in total?,I cannot answer the question about how many words Hugo spoke in total based on the available transcript data.,SELECT SUM(word_count) AS total_words FROM transcript_segments WHERE speaker = 'Hugo',True,Correcr,Incorrect,Fail,The query requires aggregating word_count for a specific speaker from the database.,"The SQL WHERE clause uses speaker = 'Hugo', which is the wrong format. It should likely be speaker = 'hugo bowne-anderson' to match the database, causing the query to return null.",The system failed to answer a valid question because the incorrect speaker name in the SQL led to no results being found.
4,Find segments mentioning 'evaluation' and provide timestamps.,"Here are the segments that mention 'evaluation' along with their timestamps:\n\n1. **Timestamp:** 12:01 - 12:28 \n **Text:** ""the foundations of Llm software and evaluation driven development...""\n\n2. **Timestamp:** 13:29 - 13:55 \n **Text:** ""...observability, testing, and evaluation, wh...","SELECT start_time_seconds, end_time_seconds, text FROM transcript_segments WHERE text LIKE '%evaluation%'",True,Correct,Correct,Pass,"The query explicitly asks to find segments mentioning a keyword and associated data (timestamps), which requires querying the database.","The SQL query correctly selects the necessary columns (start_time_seconds, end_time_seconds, text) and filters by the keyword (LIKE '%evaluation%') to retrieve the required data.","The answer accurately extracts the timestamps and text from the tool results and

## 7. Next Steps: Building the Evaluation Harness

We now have the final `df_eval` DataFrame containing the consolidated inputs, agent outputs, and human annotations (labels + explanations) for each interaction.

The next phase is to build the evaluation harness, likely in a **new notebook or script**, using the Phoenix Experiments framework:

1.  **Prepare Dataset:** Convert `df_eval` into a Phoenix Dataset, defining inputs (query, agent answer, SQL) and expected outputs (human labels, explanations).
2.  **Define Evaluators:** Create LLM-as-judge functions (using `phoenix.evals.llm_classify`) for Tool Usage, SQL Correctness, and Final Answer Quality, referencing the human explanations.
3.  **Run Experiment:** Use `phoenix.experiments.run_experiment` to apply these evaluators to the dataset.
4.  **Analyze Results:** Examine the evaluation results logged in the Phoenix UI.

In [15]:
import phoenix as px  # Add this import
px_client = px.Client() # Add this line to initialize the client

/Users/hba/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- Prepare and Upload Evaluation Dataset ---
from datetime import datetime # Ensure datetime is imported

# Check if the Phoenix client object exists. It should have been created
# in an early cell when importing libraries (e.g., px_client = px.Client())
if 'px_client' not in locals() or px_client is None:
     print("ERROR: Phoenix client 'px_client' not found or not initialized.")
     print("Please ensure you have a cell running 'px_client = px.Client()' successfully near the start of the notebook.")
     # Optionally raise an error:
     # raise NameError("Phoenix client 'px_client' is not defined.")
else:
    # Define input keys (what the agent produced + original query)
    input_keys = [
        'user_query',
        'final_answer',
        'generated_sql',
        'tool_called'
    ]

    # Define output keys (human annotations - our ground truth)
    output_keys = [
        'tool_usage_correctness_label',
        'sql_correctness_label',
        'final_answer_quality_label',
        'tool_usage_explanation',
        'sql_correctness_explanation',
        'final_answer_explanation'
    ]

    # Optional: Include original span indices for reference? Can be useful for debugging.
    # Let's add them as metadata keys instead of input/output.
    metadata_keys = ['root_span_idx', 'llm_span_idx']

    # Create a timestamp for the dataset name
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    dataset_name = f"transcript-agent-eval-data-{now}"

    print(f"Uploading DataFrame to Phoenix as dataset: {dataset_name}")

    # Ensure required columns exist before uploading
    missing_input = [k for k in input_keys if k not in df_eval.columns]
    missing_output = [k for k in output_keys if k not in df_eval.columns]
    missing_metadata = [k for k in metadata_keys if k not in df_eval.columns]

    if missing_input or missing_output or missing_metadata:
        print("ERROR: One or more specified keys are missing from df_eval!")
        print(f" Missing Input Keys: {missing_input}")
        print(f" Missing Output Keys: {missing_output}")
        print(f" Missing Metadata Keys: {missing_metadata}")
        # Optionally raise an error or stop
        raise KeyError("Missing required columns for dataset upload.")
    else:
        try:
            # Upload the dataset
            evaluation_dataset = px_client.upload_dataset(
                dataframe=df_eval,
                dataset_name=dataset_name,
                input_keys=input_keys,
                output_keys=output_keys,
                metadata_keys=metadata_keys # Add metadata keys here
            )
            print(f"Dataset uploaded successfully!")
            print(f"Dataset object type: {type(evaluation_dataset)}")



        except Exception as e:
            print(f"Error uploading dataset: {e}")


Uploading DataFrame to Phoenix as dataset: transcript-agent-eval-data-20250428-102511
📤 Uploading dataset...
💾 Examples uploaded: https://app.phoenix.arize.com/datasets/RGF0YXNldDo0/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Nw==
Dataset uploaded successfully!
Dataset object type: <class 'phoenix.experiments.types.Dataset'>

Link to view the dataset in Phoenix UI:
Error uploading dataset: 'Dataset' object has no attribute 'url'


---

**Note on Dataset Size Discrepancy:**

While our local `df_eval` DataFrame was successfully created with all 19 evaluation examples, the dataset uploaded to the Phoenix UI (`transcript-agent-eval-data-...`) currently shows only 17 examples.

The reason for this difference is unclear – it might be related to the upload process or the UI display.

For the next steps in building the evaluation harness, we will proceed using the **17 examples** available in the uploaded Phoenix dataset.

---